<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/main/recipes/geojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting GeoJSON data to Kili label format

In [ ]:
%pip install kili wget

In [ ]:
from kili.client import Kili

In [ ]:
kili = Kili()

## Data

https://geojson.io/#map=11.98/52.2103/4.383

In [ ]:
!wget https://github.com/mommermi/geotiff_sample/raw/master/sample.tif

--2023-07-12 15:16:51--  https://github.com/mommermi/geotiff_sample/raw/master/sample.tif
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mommermi/geotiff_sample/master/sample.tif [following]
--2023-07-12 15:16:51--  https://raw.githubusercontent.com/mommermi/geotiff_sample/master/sample.tif
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6018396 (5,7M) [application/octet-stream]
Saving to: ‘sample.tif’

sample.tif          100%[===================>]   5,74M  --.-KB/s    in 0,1s    

2023-07-12 15:16:52 (54,8 MB/s) - ‘sample.tif’ saved [6018396/6018396]



In [ ]:
json_interface = {}

In [ ]:
project_id = kili.create_project(
    input_type="IMAGE", title="[Kili SDK Notebook]: Geojson tutorial", json_interface=json_interface
)["id"]

In [ ]:
kili.append_many_to_dataset(project_id, content_array=["sample.tif"], external_id_array=["sample"])



INFO:kili.services.asset_import.base:Import of assets is taking a long time to complete. This maybe be due to files being processed by the server.
  0%|          | 0/1 [03:42<?, ?it/s]

100%|██████████| 1/1 [00:05<00:00,  5.54s/it]


{'id': 'cljzqwuhr01y70j2aeulw5hjf'}